In [2]:
import os
import zipfile
from io import BytesIO
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import pandas as pd
from torchvision import transforms

In [11]:
class PalmDataset(Dataset):
    def __init__(self, csv_file, zip_file, transform=None):
        self.labels = pd.read_csv(csv_file)
        self.transform = transform
        self.zip_file = zip_file

        # Открываем ZIP-файл
        self.archive = zipfile.ZipFile(zip_file, 'r')

        # Фильтрация данных только для ладоней
        self.labels = self.labels[self.labels['aspectOfHand'].str.contains('palmar', case=False, na=False)]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Получаем имя изображения
        img_name = self.labels.iloc[idx, 7]  # imageName
        img_path = f"Hands/{img_name}"  # Путь к файлу внутри архива, если все изображения в папке "images"

        # Извлекаем изображение из архива
        with self.archive.open(img_path) as img_file:
            img = Image.open(BytesIO(img_file.read())).convert("RGB")

        # Применяем трансформации, если они заданы
        if self.transform:
            img = self.transform(img)

        # Получаем метки
        age = self.labels.iloc[idx, 1]  # возраст
        skin_color = self.labels.iloc[idx, 3].lower()  # цвет кожи
        accessories = self.labels.iloc[idx, 4]  # наличие аксессуаров

        skin_color_mapping = {'very fair': 0,'fair': 1, 'medium': 2, 'dark': 3} #['fair' 'dark' 'medium' 'very fair']
        skin_color_label = skin_color_mapping.get(skin_color, -1)

        # Преобразование меток в числовой формат (например, one-hot)
        label = torch.tensor([age, skin_color_label, accessories], dtype=torch.float32)

        return img, label

In [4]:
# Путь к CSV файлу с метками и путь к ZIP-файлу с изображениями
csv_file = '/content/dataset/HandInfo.csv'
zip_file = '/content/dataset/images/Hands.zip'

# Определяем преобразования для изображений
transform = transforms.Compose([
    transforms.Resize((512, 512)),  # Изменение размера изображения до 512x512
    transforms.ToTensor(),  # Преобразование изображения в тензор
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Нормализация
])

# Создаем датасет с использованием класса PalmDataset
dataset = PalmDataset(csv_file=csv_file, zip_file=zip_file, transform=transform)

# Создаем DataLoader
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)


In [5]:
class ConditionalGenerator(nn.Module):
    def __init__(self, latent_dim, condition_dim):
        super(ConditionalGenerator, self).__init__()

        # Генератор принимает на вход шумовой вектор + вектор признаков
        self.fc = nn.Sequential(
            nn.Linear(latent_dim + condition_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512 * 512 * 3),
            nn.Tanh()  # Изображения от -1 до 1
        )

    def forward(self, z, condition):
        x = torch.cat([z, condition], dim=1)  # Объединение шума и признаков
        img = self.fc(x)
        img = img.view(-1, 3, 512, 512)  # Преобразование в изображение
        return img

In [6]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(512 * 512 * 3, 1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 1),
            nn.Sigmoid()  # Выходное значение - вероятность того, что изображение реальное
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)  # Преобразование в одномерный вектор
        validity = self.model(img_flat)
        return validity


In [ ]:
# Определение устройства (GPU или CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Параметры GAN
latent_dim = 100  # Размер шумового вектора
condition_dim = 3  # Возраст, цвет кожи, аксессуары
lr = 0.0002
num_epochs = 100

# Инициализация модели
generator = ConditionalGenerator(latent_dim, condition_dim).to(device)
discriminator = Discriminator().to(device)

# Оптимизаторы
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr)
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr)

# Функция потерь
adversarial_loss = nn.BCELoss()

# Тренировочный цикл
for epoch in range(num_epochs):
    for i, (real_imgs, labels) in enumerate(dataloader):

        # Метки для настоящих и фейковых изображений
        valid = torch.ones(real_imgs.size(0), 1).to(device)
        fake = torch.zeros(real_imgs.size(0), 1).to(device)

        # Настоящие изображения
        real_imgs = real_imgs.to(device)
        labels = labels.to(device)  # Метки: возраст, цвет кожи, аксессуары

        # === Тренировка дискриминатора ===

        # Генерация шума и фейковых изображений
        z = torch.randn(real_imgs.size(0), latent_dim).to(device)
        gen_imgs = generator(z, labels)

        # Рассчитываем потери дискриминатора на реальных и фейковых изображениях
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        # Обновляем дискриминатор
        optimizer_D.zero_grad()
        d_loss.backward()
        optimizer_D.step()

        # === Тренировка генератора ===

        # Теперь дискриминатор оценивает фейковые изображения как "настоящие"
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        # Обновляем генератор
        optimizer_G.zero_grad()
        g_loss.backward()
        optimizer_G.step()

    print(f"Epoch {epoch}/{num_epochs} | D loss: {d_loss.item()} | G loss: {g_loss.item()}")
